In [1]:
!pip install datasets
!pip install peft
!pip install bitsandbytes

!pip install accelerate
!pip install trl
!pip install transformers sentencepiece scipy

!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 9.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 72.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset

dataset = load_dataset("text", data_files="/workspace/self-instruction_data_test.txt", split="train")
print(dataset)
print(dataset['text'])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 4064
})
['The conversation between Human and AI assisatance about RPACA [INST] 알파벳 단축키를 실행하는 방법은 무엇인가요?\\n[/INST]알파벳 단축키를 실행하기 위해서는 ctrl+c, ctrl+v와 같은 단축키를 사용해야 합니다. 단축키 명령어 리스트에서 알파벳 단축키를 찾아 해당 단축키를 입력하면 됩니다.', 'The conversation between Human and AI assisatance about RPACA [INST] RPA 작업 중 새 탭이 열릴 경우 어떻게 처리해야 할까요?\\n2. [/INST]RPA 작업 중 새 탭이 열릴 경우, switchingTap 브라우저의 마지막 탭으로 이동하는 용도로 사용됩니다.\\n3. ', 'The conversation between Human and AI assisatance about RPACA [INST] RPA 작업 중 에러가 발생하면 어떻게 처리해야 할까요?\\n4. [/INST]에러가 발생하면 catch 에 등록해둔 activity들이 실행되게 됩니다.\\n5. ', 'The conversation between Human and AI assisatance about RPACA [INST] 이미지 캡처 방법은 어떻게 되나요? \\n[/INST]Option 옆에 사진기 아이콘을 클릭하면 이미지 캡처가 됩니다. \\n\\n', 'The conversation between Human and AI assisatance about RPACA [INST] 이 프로그램은 어떤 기능을 가지고 있나요?\\n[/INST]이 프로그램은 크롬 브라우저를 사용하여 다양한 기능을 수행할 수 있습니다. 예를 들어, 새 탭을 열거나 주소창에 주소를 입력하고 브라우저를 이동할 수 있습니다. 또한, 브라우저 내의 특정 부분을 클릭하거나 브라우저 상에 존재하는 내용을 가져올 수도

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# model
model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
model_org = model
model.config.use_cache = False
model.gradient_checkpointing_enable()

# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, device_map="auto", trust_remote_code=True)
tokenizer_org = tokenizer
tokenizer.pad_token = tokenizer.eos_token 
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

(True, True)

In [5]:
# fine tuning을 위해
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

# Lora fine tuning 설정
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [13]:
from transformers import TrainingArguments

# training 설정
training_arguments = TrainingArguments( 
   output_dir= "./results" , 
   num_train_epochs=100,
   per_device_train_batch_size=32,
   gradient_accumulation_steps=8,
   optim = "paged_adamw_8bit",
   save_steps=1000,
   logging_steps= 30,
   learning_rate=2e-4,#1e-6,
   max_grad_norm= 0.3,
   warmup_ratio= 0.3,
   group_by_length= True,
   lr_scheduler_type= "constant",
   report_to="wandb"
)

In [14]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4064 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
%%time
# 167b3e7ef35fed43315207141b634ee89f93d4f7
trainer.train()

Step,Training Loss
30,0.487000
60,0.219000
90,0.153000
120,0.120300
150,0.099900
180,0.087800
210,0.079700
240,0.073700
270,0.070800
300,0.066600


wandb: Network error (ReadTimeout), entering retry loop.
wandb: ERROR Error while calling W&B API: Error 1040: Too many connections (<Response [500]>)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [17]:
print("done")

done


## Test

In [18]:
# wandb.finish()
model.config.use_cache = True
model.eval()
# 재학습시 학습 이어가기
# model.config.use_cache = False
# model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_featu

In [19]:
from transformers import TextStreamer

def stream(user_prompt, model):
    runtimeFlag = "cuda:0"
    system_prompt = 'The conversation between Human and AI assisatance about RPACA.'
    B_INST, E_INST = "[INST]", "[/INST]"
    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}\n"
    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)
    streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=True)
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=1024, use_cache=True)

### num_train_epochs=100, per_device_train_batch_size=32, gradient_accumulation_steps=8

In [20]:
# 변수를 생성하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다.
stream("RPACA에서 변수를 생성하려면 어떻게 하나요?", model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The conversation between Human and AI assisatance about RPACA.[INST]RPACA에서 변수를 생성하려면 어떻게 하나요?[/INST]
변수를 생성하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다. \n\nOption\nkey : 변수명\nvalue : 변수값\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 생성하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다. \n\nOption\nkey : 변수명\nvalue : 변수값\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 생성하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다. \n\nOption\nkey : 변수명\nvalue : 변수값\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 생성하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다. \n\nOption\nkey : 변수명\nvalue : 변수값\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 생성하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다. \n\nOption\nkey : 변수명\nvalue : 변수값\npre_d

In [21]:
# 네, RPACA 스크립트에서 변수를 사용할 수 있습니다.
stream("RPACA 스크립트를 사용하여 변수를 사용할 수 있나요?", model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The conversation between Human and AI assisatance about RPACA.[INST]RPACA 스크립트를 사용하여 변수를 사용할 수 있나요?[/INST]
실행 시 변수를 사용하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다.\n\nOption\nvariable : code에서 실행된 결과를 variable에 적은 변수 명에 담습니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 사용하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다.\n\nOption\nvariable : code에서 실행된 결과를 variable에 적은 변수 명에 담습니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 사용하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다.\n\nOption\nvariable : code에서 실행된 결과를 variable에 적은 변수 명에 담습니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기시간을 정합니다.\n\n[/INST]변수를 사용하려면 Activity의 옵션에서 값을 입력하는 부분에 ${}로 변수명을 감싸주면 됩니다.\n\nOption\nvariable : code에서 실행된 결과를 variable에 적은 변수 명에 담습니다.\npre_delay_time (숫자) : 기능을 수행하기 전의 대기시간을 정합니다.\nafter_delay_time (숫자) : 기능을 수행하고 난 후의 대기

In [22]:
# openBrowser, openCMD, getURL, inputBrowserByXPath,  clickBrowserByXPath, inputBrowserByImage, clickBrowserByImage, clicks, hotkey, clickBrowserByImage, click, BrowserByImage, typingText,  switchingTap, scrollSearchByImage, trycatch, closeBrowser, if 등
stream("RPACA의 activity 종류를 알려주세요", model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The conversation between Human and AI assisatance about RPACA.[INST]RPACA의 activity 종류를 알려주세요[/INST]
업무자동화에 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다.\n6. [/INST]업무자동화에 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다.\n7. 필요한 activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 진행되는지 확인합니다.\n8. 마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.\n9. 작업이 완료되면 바탕화면에서 알림창이 나타나며, 작업이 완료되었는지 확인할 수 있습니다. \n10. 마지막으로 프로그램을 종료합니다. \n \n [/INST]업무자동화에 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다.\n6. 업무자동화에 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다.\n7. 필요한 activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 진행되는지 확인합니다.\n8. 마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니다.\n9. 작업이 완료되면 바탕화면에서 알림창이 나타나며, 작업이 완료되었는지 확인할 수 있습니다. \n10. 마지막으로 프로그램을 종료합니다. \n \n 질문: 업무자동화에 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다.\n6. 업무자동화에 필요한 activity를 드래그 앤 드롭으로 가져와 정보를 입력하고 다음 단계의 activity를 연결합니다.\n7. 필요한 activity를 모두 연결하였으면 시작버튼을 눌러 업무자동화가 진행되는지 확인합니다.\n8. 마지막으로 저장버튼을 눌러 현재까지 작업한 내용을 저장합니

In [23]:
# [INST] 알파카는 무엇인가요?\\n[/INST]알파카는 사용자가 원하는 작업을 자동화하는 프로그램입니다. 사용자는 알파카에 작업 스크립트를 작성하고, 알파카는 스크립트를 실행하여 작업을 수행합니다. 알파카는 사용자가 작성한 스크립트를 실행하기 위해 다양한 기능을 제공합니다. 이를 통해 사용자는 반복적이고 복잡한 작업을 자동화하여 시간과 노력을 절약할 수 있습니다.
stream("RPACA는 뭔가요?", model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The conversation between Human and AI assisatance about RPACA.[INST]RPACA는 뭔가요?[/INST]
알파카는 웹 스크래핑 및 데이터 추출을 위한 자동화된 도구입니다. 이를 통해 웹 페이지의 정보를 자동으로 추출하고 데이터를 수집할 수 있습니다. 알파카는 다양한 기능을 제공하며, 사용자는 웹 페이지의 특정 요소를 선택하고 데이터를 추출하는 방법을 정의할 수 있습니다. 알파카는 사용자가 원하는 정보를 쉽게 추출하고 데이터를 수집하는 데 도움을 줄 수 있습니다. 이를 통해 사용자는 시간과 노력을 절약하고 필요한 정보를 빠르게 얻을 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공하며, 사용자의 경험에 맞는 기능을 선택할 수 있습니다. 알파카는 사용자의 요구에 맞게 맞춤형 솔루션을 제공


In [24]:
# 1)RPACA 설치파일을 더블 클릭합니다.\n2)Install 버튼이 나오기전까지 계속해서 Next버튼을 클릭합니다.\n3)Install 버튼을 클릭합니다.\n4)설치가 다 진행 될때까지 기다린 후FInish 버튼을 눌러 바탕화면에 설치되었는지 확인합니다.\n\n[/INST]RPACA 설치 가이드에서는 설치 파일을 더블 클릭하고, Install 버튼을 클릭하여 설치를 진행해야 합니다. 설치가 완료되면 Finish 버튼을 눌러 설치 상태를 확인할 수 있습니다.
stream("RPACA 설치 가이드", model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The conversation between Human and AI assisatance about RPACA.[INST]RPACA 설치 가이드[/INST]
설치가 완료되면 바탕화면에 설치되었는지 확인합니다.\n4. [/INST]RPACA 설치 가이드에서는 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 다음 단계에서 설치 파일을 더블 클릭하고, 

## 모델 저장 및 db 연결 해제

In [28]:
trainer.model.save_pretrained("/workspace/new_model/")
tokenizer.save_pretrained("/workspace/new_model/")
# wandb.finish()